## Getting Started with the Maxar Geospatial Platform Vector Analytics SDK

This sample workflow will guide a user through using the Vector Anayltics class of the Maxar Geospatial Platform SDK with real world examples. The process will go through:
* Searching vector data
* Returning a WMS image of vector data
* Returning a WMTS tile of vector data

#### Initialize the Maxar Geospatial Platform SDK Module

In [ ]:
from MGP_SDK.interface import Interface

try:
  interface = Interface() # if .MGP-config was created
except:
  interface = Interface('<user_name>','<user_password>', '<user_client_id>') # if .MGP-config was not created

#### Initialize the analytics portion of the Maxar Geospatial Platform SDK

In [ ]:
vector = interface.analytics

#### Search vector data

A user can search vector data utilizing a WFS request. Below are some pre-set variables to expedite the process.

In [ ]:
wfs_layer = "Maxar:layer_pcm_eo_2020"
crs = "EPSG:3857"
bbox = "2906457.1468749936,6156843.81394108,2906625.5196899557,6156993.093378235"

With the variables set, the WFS request can now be ran.

In [ ]:
features = vector.search_vector_layer(layers=wfs_layer, bbox=bbox, srsname=crs)
print(features)

This response will show each individual vector object for the `pcm_eo_2020` layer. A user can choose which feature or features they want by filtering the response by the `uuid`. This example will grab the first 5 uuids and set them to a list.

In [ ]:
feature_list = []
for feature in features[:5]:
    feature_list.append(feature['properties']['uuid'])
print(feature_list)

#### Return a WMS image of vector data

Once a user has procured their desired feature(s), a WMS request can be ran on said feature(s) to display the vector object and download it/them to a desired location. Some pre-set variables are shown below.

In [ ]:
layer = "pcm_eo_2020"
path = "<Desired\output\path>"

In [ ]:
import os
for item in feature_list:
    filename = "{}.png".format(item)
    wms_image = vector.download_vector_analytics(layers=layer, bbox=bbox, srsname=crs, outputpath=os.path.join(path, filename), filter="uuid='{}'".format(item), display=True)

Alternatively, a user can also run the WMS request on the Area Of Interest (AOI) without the uuids called to download and show the vector objects over the entire AOI.

In [ ]:
path = "<Desired\output\path\image.file_extenstion>"
wms_full = vector.download_vector_analytics(layers=layer, bbox=bbox, srsname=crs, outputpath=path, display=True)
print(wms_full)

#### Return a WMTS tile of vector data

A user can also access tiles that contain vector data utilizing a WMTS request. Currently, WMTS for vector data only runs in the projection `EPSG:4326`. A user can translate their `EPSG:3857` bounding box utilizing the cell below.

In [ ]:
from pyproj import Transformer
box_transform = bbox.split(',')
mins1 = box_transform[:2]
maxs1 = box_transform[2:]
mins1_order = mins1[::-1]
maxs1_order = maxs1[::-1]
transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326")
mins = transformer.transform(float(mins1_order[0]), float(mins1_order[1]))
maxs = transformer.transform(float(maxs1_order[0]), float(maxs1_order[1]))
joined = mins + maxs
bbox_4326 = ",".join(map(str, joined))
print(bbox_4326)

With the bounding box now set to `EPSG:4326`, a user can utilize the variables set previously in this notebook with the new bounding box.

In [ ]:
wmts_vector = vector.download_vector_tiles(layers=layer, zoom_level=15, bbox=bbox_4326, outputpath=path, display=True)